In [826]:
# imports
import pandas as pd

# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
import json
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# cnn imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
#from tensorflow.keras.layers.normalization import BatchNormalization
#from tensorflow.keras.layers.advanced_activations import LeakyReLU

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.layers import Layer
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


#from tensorflow.keras import backend as K
#from tensorflow.keras import layers
#from tensorflow.python.keras.layers import Lambda
#from tensorflow.keras.callbacks import TensorBoard
#from tensorflow.keras.backend import sparse_categorical_crossentropy
#from tensorflow.keras.layers import Dense, TimeDistributed

from sklearn.model_selection import train_test_split

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

In [827]:
# Load data
bodies_df = pd.read_csv("train_bodies.csv")
stances_df =  pd.read_csv("train_stances.csv")

# Examine data shape
bodies_df.shape
bodies_df.head()

stances_df.shape
stances_df.head()

stances_df.head()
#combine body and stance tables
combined = pd.concat([stances_df.set_index('Body ID'),bodies_df.set_index('Body ID')], axis=1, join='inner').reset_index()

combined.shape

combined.head()

combined_df_t=combined[['Headline', 'articleBody', 'Stance']]

combined_df_t.head()

# Extract column for just labels
labels_t = combined_df_t['Stance']


In [828]:
# Examine data shape
bodies_df.shape

(1683, 2)

In [829]:
stances_df.shape

(49972, 3)

In [830]:
combined_df_t.head()

,Headline,articleBody,Stance
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...,agree
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...,unrelated
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's...",disagree


In [831]:
#combined_df.to_csv("combined_stances.csv")

In [832]:
# Extract column for just labels
#labels = combined_df['Stance']

In [833]:
combined_df_t.shape

(49972, 3)

In [931]:
# Drop columns that are strings
all_data_processed = combined_df_t.drop(['Stance'], axis=1)
#all_data_processed = combined_df_t.drop(['articleBody'], axis=1)

In [835]:
new_labels1= labels_t.copy()

new_labels1[new_labels1=='agree']=1
new_labels1[new_labels1=='unrelated']=0
new_labels1[new_labels1=='disagree']=0
new_labels1[new_labels1=='discuss']=0


In [836]:
new_labels1

0        0
1        1
2        0
3        0
4        0
        ..
49967    0
49968    0
49969    1
49970    0
49971    0
Name: Stance, Length: 49972, dtype: object

In [918]:
sum(new_labels1)/len(new_labels1)

0.07360121668134155

In [932]:
all_data_processed.head()

,Headline,articleBody
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's..."


In [933]:
# Set some variables to hold test, dev, and training data.
test_data, test_labels = all_data_processed[1000:], new_labels1[1000:]
#dev_data, dev_labels = all_data_processed[17505:23340], labels[17505:23340]
train_data, train_labels = all_data_processed[:1000], new_labels1[:1000]

In [934]:
len(test_data)

48972

In [935]:
len(train_data)

1000

In [838]:
test_labels.head()

1000    0
1001    0
1002    0
1003    0
1004    0
Name: Stance, dtype: object

In [839]:
# Used GLoVE embeddings
embeddings_dict = {}
with open("glove.6B.50d.txt", 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        token = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[token] = vector

In [840]:
def sent_to_ids(sentences):
    """
    converting a list of strings to a list of lists of word ids
    """
    
    # TODO - Update Regular Expression for parsing (TINA)
    # Improvement Option (Remove trailing 's') Check Missing list for optimization
    
    #re.sub('[^A-Za-z0-9]+', '', mystring)
    #cleanString = re.sub('\W+','', string ) 
    
    bad_chars = [';', ':', '!', "*"] 
    
    text_ids = []
    example = []
    sentences = sentences.replace('.','')
    sentences = sentences.replace('\'','')
    sentences = sentences.replace(',','')
    sentences = sentences.replace('$','')
    sentences = sentences.replace('/',' ')
    sentences = sentences.replace('','')
    for word in sentences.split(' '):
        #word.lower()
        temp = word.lower()
        #temp.replace('\'','')
        if temp in embeddings_dict:
            example.append(embeddings_dict[temp])
        else:
            print("Missing ", temp)            
    text_ids.append(example)

    return  text_ids   

In [936]:
all_data_emb = all_data_processed.copy()

In [937]:
#dont run
#for i in range (1250):
#    all_data_emb.at[i, 'Headline'] =  np.array(sent_to_ids(all_data_processed.at[i,'Headline']))
#    all_data_emb.at[i, 'articleBody'] =  np.array(sent_to_ids(all_data_processed.at[i,'articleBody']))

In [938]:
#all_data_emb

,Headline,articleBody
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's..."
...,...,...
49967,Urgent: The Leader of ISIL 'Abu Bakr al-Baghda...,(CNN) -- Boko Haram laughed off Nigeria's anno...
49968,Brian Williams slams social media for speculat...,Along with unveiling the Apple Watch earlier t...
49969,Mexico Says Missing Students Not Found In Firs...,The bodies found in a mass grave were confirme...
49970,US Lawmaker: Ten ISIS Fighters Have Been Appre...,Rep. Duncan Hunter (R-CA) told Greta Van Suste...


In [939]:
all_data_processed['Headline'][1]

'Hundreds of Palestinians flee floods in Gaza as Israel opens dams'

In [940]:
train_data

,Headline,articleBody
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's..."
...,...,...
995,Weather Channel’s Mike Seidel was not caught w...,New York (CNN) -- When war reporter James Fole...
996,NYPD to issue tickets instead of arrest for we...,Small-time drug offenders in New York City cou...
997,The iPhone 6 'Hairgate' Is Not a Thing,First there was Bendgate. Then Gapgate. And no...
998,'Spiderman' squished: Perth experts debunk spi...,While the US Vogue offices are moving to 1 Wor...


In [885]:
#train_data= train_data['Headline']

In [946]:
train_data_token = train_data['Headline']+train_data['articleBody']

In [948]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(train_data_token)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(test_data)
#print(encoded_docs)

In [949]:
#no need to rerun
embeddings_index_ex = dict()
with open("glove.6B.50d.txt", 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index_ex[word] = coefs
#    f.close()
print('Loaded %s word vectors.' % len(embeddings_index_ex))

Loaded 400000 word vectors.


In [950]:
t.word_index.items()

dict_items([('the', 1), ('to', 2), ('a', 3), ('of', 4), ('in', 5), ('and', 6), ('that', 7), ('is', 8), ('on', 9), ('was', 10), ('for', 11), ('said', 12), ('with', 13), ('it', 14), ('he', 15), ('as', 16), ('have', 17), ('by', 18), ('his', 19), ('has', 20), ('from', 21), ('be', 22), ('an', 23), ('at', 24), ('not', 25), ('are', 26), ('”', 27), ('been', 28), ('but', 29), ('who', 30), ('this', 31), ('had', 32), ('they', 33), ('after', 34), ('we', 35), ('were', 36), ('one', 37), ('will', 38), ('i', 39), ('about', 40), ('which', 41), ('new', 42), ('or', 43), ('isis', 44), ('video', 45), ('she', 46), ('apple', 47), ('also', 48), ('their', 49), ('state', 50), ('more', 51), ('would', 52), ('out', 53), ('up', 54), ('her', 55), ('when', 56), ('all', 57), ('told', 58), ('no', 59), ('there', 60), ('its', 61), ('could', 62), ('people', 63), ('year', 64), ('what', 65), ('him', 66), ('you', 67), ('man', 68), ('al', 69), ('islamic', 70), ('some', 71), ('group', 72), ('if', 73), ('news', 74), ('time', 75

In [951]:
# create a weight matrix for words in training docs
embedding_matrix_ex = np.zeros((vocab_size, 50))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index_ex.get(word)
    if embedding_vector is not None:
        embedding_matrix_ex[i] = embedding_vector

In [952]:
vocab_size

18474

In [953]:
train_data

,Headline,articleBody
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's..."
...,...,...
995,Weather Channel’s Mike Seidel was not caught w...,New York (CNN) -- When war reporter James Fole...
996,NYPD to issue tickets instead of arrest for we...,Small-time drug offenders in New York City cou...
997,The iPhone 6 'Hairgate' Is Not a Thing,First there was Bendgate. Then Gapgate. And no...
998,'Spiderman' squished: Perth experts debunk spi...,While the US Vogue offices are moving to 1 Wor...


In [972]:
num_steps = 1
hidden_size = 10

model2 = Sequential()
#model2.add(Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32'))
#model.add(Embedding(vocab, hidden_size, input_length=num_steps))

#model2.add(Embedding(vocab_size, hidden_size, input_length=num_steps))
model2.add(Embedding(vocab_size, 50, weights=[embedding_matrix_ex], input_length=8, trainable=False))

#model2.add(LSTM(hidden_size, return_sequences=True))
#model2.add(LSTM(hidden_size, return_sequences=True))

#model.add(Conv2D(64, (3, 3)))

model2.add(Flatten())
#model2.add(Dense(1, activation='sigmoid'))
model2.add(Dense(1))
model2.add(Activation('sigmoid'))


#model2.add(Dropout(0.5))
#model.add(TimeDistributed(Dense(vocab_size)))
#model2.add(Activation('softmax'))

In [973]:
# summarize the model
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_53 (Embedding)     (None, 8, 50)             923700    
_________________________________________________________________
flatten_29 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 401       
_________________________________________________________________
activation_33 (Activation)   (None, 1)                 0         
Total params: 924,101
Trainable params: 401
Non-trainable params: 923,700
_________________________________________________________________
None


In [981]:
def encodeDoc(data):
# integer encode the documents
#encoded_docs = t.texts_to_sequences(all_data_processed['Headline'])
#encoded_docs = t.texts_to_sequences(train_data)
    encoded_docs = t.texts_to_sequences(data)

    #print(encoded_docs)
    # pad documents to a max length of 4 words
    max_length = 4
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    #print(padded_docs)
    
    return padded_docs

In [982]:
def combineEncode(data):
    padded_docs_h = encodeDoc(data['Headline'])
    padded_docs_b = encodeDoc(data['articleBody'])
    padded_docs=np.concatenate((padded_docs_h, padded_docs_b), axis=1)
    return padded_docs

In [956]:
#headline
#padded_docs_h = encodeDoc(train_data['Headline'])
#print(padded_docs)

In [957]:
#body
#padded_docs_b = encodeDoc(train_data['articleBody'])

In [966]:
#padded_docs=np.concatenate((padded_docs_h, padded_docs_b), axis=1)

In [983]:
padded_docs=combineEncode(train_data)

In [984]:
padded_docs[0]

array([1998,   34,   99, 4570,    1,  361,   47, 4344])

In [975]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [976]:
# fit the model
batch_size = 30
model2.fit(padded_docs, train_labels, batch_size=batch_size, epochs=50)


Epoch 1/50
1000/1000 [==============================] - 3s 3ms/step - loss: 0.7473 - acc: 0.5590
Epoch 2/50
1000/1000 [==============================] - 0s 143us/step - loss: 0.3818 - acc: 0.8900
Epoch 3/50
1000/1000 [==============================] - 0s 134us/step - loss: 0.3380 - acc: 0.9060
Epoch 4/50
1000/1000 [==============================] - 0s 137us/step - loss: 0.3259 - acc: 0.9070
Epoch 5/50
1000/1000 [==============================] - 0s 129us/step - loss: 0.3164 - acc: 0.9080
Epoch 6/50
1000/1000 [==============================] - 0s 141us/step - loss: 0.3073 - acc: 0.9080
Epoch 7/50
1000/1000 [==============================] - 0s 165us/step - loss: 0.2993 - acc: 0.9090
Epoch 8/50
1000/1000 [==============================] - 0s 137us/step - loss: 0.2916 - acc: 0.9100
Epoch 9/50
1000/1000 [==============================] - 0s 130us/step - loss: 0.2839 - acc: 0.9100
Epoch 10/50
1000/1000 [==============================] - 0s 144us/step - loss: 0.2769 - acc: 0.9110
Epoch 11/50

In [985]:
encodedTestData = combineEncode(test_data)

In [986]:
results = model2.predict(encodedTestData)

In [988]:
# evaluate the model
#loss, accuracy =
accuracy = model2.evaluate(encodedTestData, test_labels, verbose=1)
print('Loss: %d' % (accuracy[0]*100))
print('Accuracy: %f' % (accuracy[1]*100))

48972/48972 [==============================] - 4s 88us/step
Loss: 32
Accuracy: 90.541534


In [989]:
results[:20]

array([[0.02518052],
       [0.03751662],
       [0.04014981],
       [0.01169181],
       [0.14870054],
       [0.01645923],
       [0.06975237],
       [0.10703376],
       [0.09786105],
       [0.28896138],
       [0.04295948],
       [0.03574237],
       [0.12162167],
       [0.0248453 ],
       [0.6106898 ],
       [0.02393684],
       [0.01995066],
       [0.00540704],
       [0.06817186],
       [0.03356746]], dtype=float32)

In [990]:
new_labels1[:20]

0     0
1     1
2     0
3     0
4     0
5     1
6     0
7     0
8     1
9     0
10    0
11    1
12    0
13    0
14    0
15    0
16    0
17    1
18    0
19    0
Name: Stance, dtype: object

In [991]:
num_steps = 1
hidden_size = 35

model3 = Sequential()
#model2.add(Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32'))
#model.add(Embedding(vocab, hidden_size, input_length=num_steps))

#model2.add(Embedding(vocab_size, hidden_size, input_length=num_steps))
model3.add(Embedding(vocab_size, 50, weights=[embedding_matrix_ex], input_length=8, trainable=False))

model3.add(LSTM(hidden_size, return_sequences=True))
model3.add(LSTM(hidden_size, return_sequences=True))

#model.add(Conv2D(64, (3, 3)))

model3.add(Flatten())
#model2.add(Dense(1, activation='sigmoid'))
#model3.add(Dense(20))
model3.add(Dense(units=20, input_dim=35, activation='relu')) 
model3.add(Dropout(0.5))
#model3.add(Flatten())
model3.add(Dense(units=1, input_dim=20, activation='relu')) 

#model3.add(Activation('sigmoid'))


#model2.add(Dropout(0.5))
#model.add(TimeDistributed(Dense(vocab_size)))
#model2.add(Activation('softmax'))

In [992]:
# summarize the model
print(model3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_54 (Embedding)     (None, 8, 50)             923700    
_________________________________________________________________
lstm_69 (LSTM)               (None, 8, 35)             12040     
_________________________________________________________________
lstm_70 (LSTM)               (None, 8, 35)             9940      
_________________________________________________________________
flatten_30 (Flatten)         (None, 280)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 20)                5620      
_________________________________________________________________
dropout_31 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 21        
Total para

In [993]:
model3.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
#model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [994]:
# fit the model
batch_size = 30
model3.fit(padded_docs, train_labels, batch_size=batch_size, epochs=50)

Epoch 1/50
1000/1000 [==============================] - 5s 5ms/step - loss: 1.4511 - acc: 0.9100A: 2s - loss: 1.4957 - acc: 
Epoch 2/50
1000/1000 [==============================] - 1s 1ms/step - loss: 1.4506 - acc: 0.9100
Epoch 3/50
1000/1000 [==============================] - 1s 1ms/step - loss: 1.4506 - acc: 0.9100
Epoch 4/50
1000/1000 [==============================] - 1s 1ms/step - loss: 1.4506 - acc: 0.9100
Epoch 5/50
1000/1000 [==============================] - 1s 1ms/step - loss: 1.4506 - acc: 0.9100
Epoch 6/50
1000/1000 [==============================] - 1s 1ms/step - loss: 1.4506 - acc: 0.9100
Epoch 7/50
1000/1000 [==============================] - 1s 1ms/step - loss: 1.4506 - acc: 0.9100
Epoch 8/50
1000/1000 [==============================] - 1s 1ms/step - loss: 1.4506 - acc: 0.9100
Epoch 9/50
1000/1000 [==============================] - 1s 1ms/step - loss: 1.4506 - acc: 0.9100
Epoch 10/50
1000/1000 [==============================] - 1s 1ms/step - loss: 1.4506 - acc: 0.9100
E

In [995]:
# evaluate the model
#loss, accuracy =
accuracy = model3.evaluate(encodedTestData, test_labels, verbose=1)
print('Loss: %d' % (accuracy[0]*100))
print('Accuracy: %f' % (accuracy[1]*100))

48972/48972 [==============================] - 26s 530us/step
Loss: 118
Accuracy: 92.673364


In [996]:
results[:20]

array([[0.02518052],
       [0.03751662],
       [0.04014981],
       [0.01169181],
       [0.14870054],
       [0.01645923],
       [0.06975237],
       [0.10703376],
       [0.09786105],
       [0.28896138],
       [0.04295948],
       [0.03574237],
       [0.12162167],
       [0.0248453 ],
       [0.6106898 ],
       [0.02393684],
       [0.01995066],
       [0.00540704],
       [0.06817186],
       [0.03356746]], dtype=float32)

In [997]:
new_labels1[:20]

0     0
1     1
2     0
3     0
4     0
5     1
6     0
7     0
8     1
9     0
10    0
11    1
12    0
13    0
14    0
15    0
16    0
17    1
18    0
19    0
Name: Stance, dtype: object